In [ ]:
import numpy as np


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from xgboost import plot_importance

In [ ]:
from sklearn.preprocessing import StandardScaler as ss

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
from xgboost.sklearn import XGBClassifier

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import auc, roc_curve
from sklearn.metrics import confusion_matrix

In [ ]:
from sklearn.model_selection import cross_val_score
from bayes_opt import BayesianOptimization

In [ ]:
import eli5
from eli5.sklearn import PermutationImportance

In [ ]:
import time
import os
import gc
import random
from scipy.stats import uniform

In [ ]:
os.chdir("../input/")

In [ ]:
data = pd.read_csv("winequalityN.csv")

In [ ]:
data.head()


In [ ]:
data.info()


In [ ]:
data.shape

In [ ]:
data.describe

In [ ]:
data.isnull().values.any()


In [ ]:
data.isnull().sum()


In [ ]:
data.dropna(axis=0,inplace=True)

In [ ]:
data.dropna(axis=0,inplace=True)

In [ ]:
data.shape

In [ ]:
sns.pairplot(data, diag_kind='scatter',hue='type')


In [ ]:
X = data.iloc[ :, 1:14]

In [ ]:
y=data.iloc[:,0]

In [ ]:
y = y.map({'white':1, 'red' : 0})


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.35,
                                                    shuffle = True
                                                    )

In [ ]:
pipe_list=[('scale',ss()),
           ('pca',PCA()),
		   ('xgb',XGBClassifier(n_jobs=2,
							    silent=False))
							 ]

In [ ]:
pipe=Pipeline(pipe_list)

In [ ]:
grid_parameters={'xgb__learning_rate':  [0.05, 0.07],
              'xgb__n_estimators':   [50,  100],
              'xgb__max_depth':      [3,5],
              'pca__n_components' : [5,7]
              }

In [ ]:
grid_search=GridSearchCV(pipe,
						  grid_parameters,
						   cv=3,
						   n_jobs=3,
						   verbose=1,
						   scoring=['accuracy','roc_auc'],
						   refit='roc_auc')

In [ ]:
start=time.time()
grid_search.fit(X_train,y_train)
stop=time.time()
(stop-start)/60

In [ ]:
f"Best score: {grid_search.best_score_} "

In [ ]:
f"Best parameter set {grid_search.best_params_}"

In [ ]:
plt.bar(grid_search.best_params_.keys(), grid_search.best_params_.values(), color='r')
plt.xticks(rotation=45)

In [ ]:
y_pred=grid_search.predict(X_test)
y_pred

In [ ]:
accuracy=accuracy_score(y_test,y_pred)

In [ ]:
f"Accuracy is{accuracy*100}%"

In [ ]:
parameter_random = {'xgb__learning_rate':  uniform(0, 1),
                    'xgb__n_estimators':   range(50,100),
                    'xgb__max_depth':      range(3,5),
                    'pca__n_components' : range(5,7)}

In [ ]:
random_search=RandomizedSearchCV(pipe,
                             param_distributions=parameter_random,
                             cv=3,
                             n_iter=27,
                             n_jobs=3,
                             verbose=1,
                             scoring=['accuracy','roc_auc'],
                             refit='roc_auc')

In [ ]:
start=time.time()
random_search.fit(X_train,y_train)
stop=time.time()
(stop-start)/60

In [ ]:
f"Best score: {random_search.best_score_} "

In [ ]:
f"Best parameter set {random_search.best_params_}"

In [ ]:
y_pred=random_search.predict(X_test)
y_pred

In [ ]:
accuracy=accuracy_score(y_test,y_pred)

In [ ]:
f"Accuracy is{accuracy*100}%"

In [ ]:
plt.bar(random_search.best_params_.keys(), random_search.best_params_.values(), color='g')
plt.xticks(rotation=45)

In [ ]:
parameter_bo={
           'learning_rate':  (0, 1),            
           'n_estimators':   (50,100),         
           'max_depth':      (3,5),            
           'n_components' :  (5,7)
            }